In [17]:
!pip3 install gensim==3.6.0
!pip install gensim newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13542 sha256=bb1b0f5c12576dac13567939f35ce9e74a5dca1222528f419009cad168413e2d
  Stored in directory: /home/baekkwsl/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=e392cf0ad30b32880a65c9be0c15dc7463e445e94328d7d1bf0d22777082420e
  Stored in directory: /home/baekkwsl/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933e41c4ccd6a8f8edb
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398380 sha256=efc1c28aa44e057fba05c5ed07ee9d5be4a107416f12d11522d6fe7a8f1e7788
  Stored in directory: /home/baekkwsl/.cache/pip/

In [28]:
import json
from tqdm.auto import tqdm
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
import os

In [44]:
from huggingface_hub import login

#허깅페이스 액세스토큰
hf_token = ""

# 허깅페이스 로그인
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/baekkwsl/.cache/huggingface/token
Login successful


In [45]:
# 1. JSON 데이터 로드
with open('./data/combined_data.json', 'r') as f:
    data = json.load(f)

In [46]:
# 닉네임과 《 기준 왼쪽의 공백까지 제거하는 함수 정의
def remove_nickname(text):
    # 정규 표현식으로 닉네임과 《 앞 공백까지 모두 제거
    return re.sub(r'\s*[^《]*《', '', text)

In [18]:
#gensim을 이용

from gensim.summarization import summarize

# 댓글을 요약하는 함수 정의
def summarize_comments(comments):
    # 댓글 리스트를 하나의 문자열로 합치기
    comments_text = ' '.join(comments)
    try:
        # gensim의 summarize 함수로 요약하기
        summary = summarize(comments_text, word_count=100)
    except ValueError:
        # 텍스트가 너무 짧아서 요약이 불가능한 경우 원본 반환
        summary = comments_text
    return summary

In [19]:
# 크롤링한 데이터를 학습용 데이터셋으로 바꾸기 위함

In [49]:
for item in tqdm(data[:5], desc="Processing Data", unit="item"):
    title = item.get('title', '')
    content = item.get('content', '')
    comments = item.get('comments', [])

    for com in comments:
        print(remove_nickname(com))
        print("@@@@")
    # 댓글이 리스트인지 확인하고 닉네임 제거 후 문자열로 결합
    #if isinstance(comments, list):
    #    cleaned_comments = ' '.join([remove_nickname(comment) for comment in comments])
    #else:
    #    cleaned_comments = remove_nickname(comments)

    # 결과 출력
    print(f"제목: {title}")
    print(f"질문 내용: {content}")
    print(f"요약된 답변: {cleaned_comments}")
    print('-' * 50)

Processing Data:   0%|          | 0/5 [00:00<?, ?item/s]

좋아요
@@@@
돈 좀 쓰더라도 레드..
@@@@
제목: 라이벌이탭 체인 어떤가요?
질문 내용: 소음이나 성능 어떤가요?
요약된 답변: 2021년도 4월에 출시됬는데.. TT용 디스크휠은 신형 나왔던데.비슷한 시기의 휠들도 아직 신형 안나온거같아요스페셜,트렉 등등
--------------------------------------------------
랜도너스 아니면 솔라 필요없어요1040 솔라 사용중
@@@@
1050이요 일주 같은거 하시는게 아니라면 배터리 걱정될정도 아닌긋요
@@@@
당장필요하면 1040솔라좀 더 기다리는게 가능하시면 1050솔라 존버요.솔라 추천이유는 배터리 걱정에 좀 더 자유로울거같고1050은 장착할때 어떤 핸들바에는 너무 타이트하단 말도있고해서 50솔라는 좀 더 개선해서 나오지 않을까해서요
@@@@
1050
@@@@
1040 솔라 괜찮습니다 1050 이슈도 좀 있고 중고 1040솔라 가격 많이 낮아서 괜찮을 듯 합니다
@@@@
1040 썼었는데 구지 솔라까지 필요없습니다 1040도 배터리 좀비라 ㅎㅎ 전 1040추천
@@@@
1040이랑 1040솔라랑 차이점 중 하나가 마운트 재질입니다.솔라는 마운트하는 부분이 금속이고 1040논솔라는 금속이 아닙니다.가민 1000이랑 1030쓰면서 날개파손 두번 당해보니 솔라마운트 부분 보고 뒤도 안보고 솔라로 했습니다.
@@@@
1040번들인데 금속 맞아요
@@@@
1040솔라아닌것도 금속재질입니다. 이번1050이 다시플라스틱
@@@@
핫?? 제가 잘못본거군요. 1040도 괜찮네요 ㅎㅎ 이래되면 나는 왜...
@@@@
제가 잘 못 알았네요. 구매할때 논솔라가 플라스틱으로 봤는데 눈이 삐었나 봅니다.
@@@@
1050에서 다시 금속을 배제한게부러지지말라고 금속 했으더 마운트를 파 드시는 경우가 종종 있었나봄...최근엔 모두 교체가 어렵지 않은 플라스틱류로 나오죠
@@@@
1040솔라 사용하다 1050으로 왔는데 너무 만족합니다 벨소리, 경로 소리 안내, gps정확도 1040 솔라 보다는 확실히 개